# Chapter 7 — SQL Analysis Examples (AdventureWorks2022)
This notebook contains 10 propositions, SQL queries, explanations, and a presentation script.


## Proposition: Total Sales Per Customer

### Explanation
- The query joins customers with their orders.
- Groups by customer to calculate total sales.
- Uses SUM to aggregate TotalDue values.


In [ ]:
SELECT c.CustomerID, SUM(soh.TotalDue) AS TotalSales 
FROM Sales.Customer c 
JOIN Sales.SalesOrderHeader soh 
ON c.CustomerID = soh.CustomerID 
GROUP BY c.CustomerID;

## Proposition: Top 5 Products by Revenue

### Explanation
- Aggregates revenue per product.
- Sorts in descending order to get highest revenue.
- Limits result to the top five.


In [ ]:
SELECT TOP 5 p.ProductID, SUM(sod.LineTotal) AS Revenue 
FROM Production.Product p 
JOIN Sales.SalesOrderDetail sod 
ON p.ProductID = sod.ProductID 
GROUP BY p.ProductID 
ORDER BY Revenue DESC;

## Proposition: Average Order Value Per Territory

### Explanation
- Computes average TotalDue per sales territory.
- Useful for comparing performance across regions.
- Uses AVG aggregate function.


In [ ]:
SELECT st.TerritoryID, AVG(soh.TotalDue) AS AvgOrderValue 
FROM Sales.SalesTerritory st 
JOIN Sales.SalesOrderHeader soh 
ON st.TerritoryID = soh.TerritoryID 
GROUP BY st.TerritoryID;

## Proposition: Employee Sales Count Using Window RANK

### Explanation
- Counts orders per salesperson.
- Applies RANK window function to compare salespeople.
- Ranks by total sales in descending order.


In [ ]:
SELECT sp.BusinessEntityID, COUNT(soh.SalesOrderID) AS SalesCount, RANK() 
OVER (
ORDER BY COUNT(soh.SalesOrderID) DESC) AS RankBySales 
FROM Sales.SalesPerson sp 
JOIN Sales.SalesOrderHeader soh 
ON sp.BusinessEntityID = soh.SalesPersonID 
GROUP BY sp.BusinessEntityID;

## Proposition: Running Total of Sales Per Month

### Explanation
- Groups orders by year and month.
- Calculates monthly sales totals.
- Computes a running cumulative total with a window function.


In [ ]:
SELECT 
    YEAR(OrderDate) AS Yr,
    MONTH(OrderDate) AS Mo,
    SUM(TotalDue) AS MonthlySales,
    SUM(SUM(TotalDue)) OVER (
        ORDER BY YEAR(OrderDate), MONTH(OrderDate)
    ) AS RunningTotal
FROM Sales.SalesOrderHeader
GROUP BY YEAR(OrderDate), MONTH(OrderDate);


## Proposition: Pivot Quantity Sold Per Product Category

### Explanation
- Transforms rows into columns using PIVOT.
- Aggregates quantities sold per category.
- Useful for cross-category analysis.


In [ ]:
SELECT * 
FROM ( 
SELECT pc.Name AS Category, sod.OrderQty 
FROM Production.ProductCategory pc 
JOIN Production.ProductSubcategory ps 
ON pc.ProductCategoryID = ps.ProductCategoryID 
JOIN Production.Product p 
ON ps.ProductSubcategoryID = p.ProductSubcategoryID 
JOIN Sales.SalesOrderDetail sod 
ON p.ProductID = sod.ProductID ) AS SourceTable 
PIVOT ( SUM(OrderQty) FOR Category IN ([Accessories],[Bikes],[Clothing],[Components]) ) AS Pivoted;

## Proposition: Previous Order Total Using LAG

### Explanation
- Uses LAG to access previous row data.
- Partitions by customer to track order history.
- Shows how customer spending changes over time.


In [ ]:
SELECT 
    soh.CustomerID, 
    soh.TotalDue, 
    LAG(soh.TotalDue) OVER (
        PARTITION BY soh.CustomerID
        ORDER BY soh.OrderDate
    ) AS PreviousOrder
FROM Sales.SalesOrderHeader soh;


## Proposition: First Purchase Date Per Customer

### Explanation
- Identifies earliest order date for each customer.
- Useful for cohort and retention analysis.
- Applies MIN aggregate function.


In [ ]:
SELECT CustomerID, MIN(OrderDate) AS FirstPurchase 
FROM Sales.SalesOrderHeader 
GROUP BY CustomerID;

## Proposition: Most Recent List Price Per Product

### Explanation
- Uses FIRST_VALUE to get latest price by ordering from newest to oldest.
- Partitions by product for independent histories.
- Simplifies price lookup for analytics.


In [ ]:
SELECT 
    plp.ProductID, 
    FIRST_VALUE(plp.ListPrice) OVER (
        PARTITION BY plp.ProductID 
        ORDER BY plp.StartDate DESC 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS LatestPrice
FROM Production.ProductListPriceHistory plp;


## Proposition: Orders Grouped by TotalDue Range

### Explanation
- Demonstrates grouping by CASE expressions.
- Categorizes orders into ranges.
- Useful for distribution and segmentation analysis.


In [ ]:
SELECT 
CASE 
WHEN TotalDue < 100 
THEN 'Low' 
WHEN TotalDue BETWEEN 100 AND 1000 
THEN 'Medium' 
ELSE 'High' END AS AmountRange, COUNT(*) AS OrderCount 
FROM Sales.SalesOrderHeader 
GROUP BY 
CASE 
WHEN TotalDue < 100 
THEN 'Low' 
WHEN TotalDue BETWEEN 100 AND 1000 
THEN 'Medium' 
ELSE 'High' END;